# Contents

* [Best Fitting Calculations for *Rarely Mapped 20*](#Best-Fitting-Calculations-for-Rarely-Mapped-20)
* [Covered *k*-mer Calculations for Edlib Results](#Covered-k\-mer-Calculations-for-Edlib-Results)
* [Covered *k*-mer Calculations for All Results of Competitors](#Covered-k\-mer-Calculations-for-All-Results-of-Competitors)


# Best Fitting Calculations for *Rarely Mapped 20*
Here, we parse the nested results for read set *rarely mapped 20*, find best fitting intervals for sets of nested intervals and create lists of covered *k*-mers per read

In [21]:
def loadFindThomsRes(pth, refSketch):
    compResPerRead = {}
    
    for l in open(pth, 'r'):
        l = l.strip()

        if l.startswith('s'):
            lastRdId = int(l.split('_')[1])
            compResPerRead[lastRdId] = []
        else:
            cols = l.split(' ')
            s = refSketch[int(cols[1])][0] + 1 - K
            e = refSketch[int(cols[3])][0]
            compResPerRead[lastRdId].append({"id": lastRdId, "start": s, "end": e, "score": float(cols[5])})
        
    return compResPerRead

In [41]:
def loadSingleFindThomsRes(pth, refSketch):
    res = []
    
    for l in open(pth, 'r'):
        cols = l.strip().split(' ')
        s = refSketch[int(cols[1])][0] + 1 - K
        e = refSketch[int(cols[3])][0]
        res.append({"start": s, "end": e, "score": float(cols[5])})
        
    return res

In [7]:
from collections import deque
from Bio.Seq import Seq

NT_IN_BITS = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

#This function calculates the minimizer sketch of a sequence. It is influenced by the code of "The minimizer Jaccard
#estimator is biased and inconsistent." from Belbasi et al. (function "winnowed_minimizers_linear(perm,windowSize)" 
#in file "winnowed_minimizers.py").
def calcMiniSketch(seq, k, w):
    sketch = []
    #A deque to store k-mer hashes inside the current window
    windowKmers = deque()
    mask = (4 ** k) - 1
    lastIdx = -1

    for i in range(len(seq) - k + 1):
        kmerBits = 0
        kmerBitsRevComp = 0
        windowBorder = i - (w - 1)
        
        #Get bit representation of k-mer
        for c in seq[i:i+k]:
            kmerBits = (kmerBits << 2) + NT_IN_BITS[c]

        #Get bit representation of k-mer's reverse complement
        for c in str(Seq(seq[i:i+k]).reverse_complement()):
            kmerBitsRevComp = (kmerBitsRevComp << 2) + NT_IN_BITS[c]

        #If a k-mer is its own reverse complement we skip it
        if kmerBits == kmerBitsRevComp:
            continue

        #Depending on which hash is smaller we consider either a k-mer or its reverse complement per position
        if kmerBits < kmerBitsRevComp:
            kmer = (i, kmerBits, getHash(kmerBits, mask))
        else:
            #A k-mer is a pair of k-mer's start position and its hash
            kmer = (i, kmerBitsRevComp, getHash(kmerBitsRevComp, mask))
            
        #Remove all k-mers with a hash value larger than the newly calculated one
        while (len(windowKmers) > 0) and (windowKmers[-1][2] > kmer[2]):
            windowKmers.pop()

        #Save new k-mer as window k-mer
        windowKmers.append(kmer)

        #Remove k-mer if it is not any longer inside the window
        while (len(windowKmers) > 0) and (windowKmers[0][0] < windowBorder):
            windowKmers.popleft()

        #As soon as we have seen a first full window of k-mers choose a minimizer
        if (windowBorder >= 0) and (len(windowKmers) > 0):      
            #We do not choose the same minimizer for a second time
            if lastIdx != windowKmers[0][0]:
                sketch.append((windowKmers[0][0]+k-1, windowKmers[0][1], windowKmers[0][2]))
                lastIdx = windowKmers[0][0]
                
            while len(windowKmers) > 1 and windowKmers[0][1] == windowKmers[1][1]:
                windowKmers.popleft()
                sketch.append((windowKmers[0][0]+k-1, windowKmers[0][1], windowKmers[0][2]))    
                lastIdx = windowKmers[0][0] 

    #If our sequence was too small to get a full window of k-mers to consider take the smallest one found so far
    if windowBorder < 0 and len(windowKmers) > 0:
        sketch.append((windowKmers[0][0]+k-1, windowKmers[0][1], windowKmers[0][2]))
        
        while len(windowKmers) > 1 and windowKmers[0][1] == windowKmers[1][1]:
            windowKmers.popleft()
            sketch.append((windowKmers[0][0]+k-1, windowKmers[0][1], windowKmers[0][2]))

    return sketch

In [17]:
#This function filters out best fitting intervals from a given set of homology intervals. Given a set of nested
#homology intervals, an interval is "best fitting" if its length is closest to the length of the corresponding read
#Parameter: intCoords: A dictionary with integer-based read ids as keys and lists of pairs representing start and
#                      end coordinates of intervals as values
def filterBestFittingInts(intCoords):
    bestFitting = {}
    
    #For each read...
    for r in intCoords:
        bestFitting[r] = []
        #Clusters of nested intervals are stored as a dictionary with the coordinates of the outermost interval as 
        #keys and a list of all interval pairs as values
        nestedInts = {}
        
        #...cluster nested intervals together
        for i in intCoords[r]:
            clusterFound = False
            
            for c in nestedInts:
                #Check if the current interval covers the outermost interval of this cluster
                if i[0] <= c[0] and i[1] >= c[1]:
                    nestedInts[i] = list(nestedInts[c]) + [i]
                    del nestedInts[c]
                    clusterFound = True
                    break
                #Check if the current interval is covered by the outermost interval of this cluster already
                elif i[0] >= c[0] and i[1] <= c[1]:
                    nestedInts[c].append(i)
                    clusterFound = True
                    break
                
            #Check if we could not find a matching cluster for this interval
            if not clusterFound:
                #Create a new cluster
                nestedInts[i] = [i]
            
        #Select the best fitting interval of each cluster
        readLength = len(rdSeqs[f"s_{r}"].seq)

        for c in nestedInts:
            minDiff = abs((c[1] - c[0]) - readLength)
            bestInt = c
                
            for i in nestedInts[c]:
                diff = abs((i[1] - i[0]) - readLength)
                    
                if diff < minDiff:
                    minDiff = diff
                    bestInt = i
                    
            bestFitting[r].append(bestInt)
    
    return bestFitting

In [82]:
from numpy import unique
from sys import maxsize

def filterBestFittingIntsBetter(intCoords):
    bestFitting = {}
    
    #For each read...
    for r in intCoords:
        bestFitting[r] = []
        #Clusters of nested intervals are stored as a dictionary with the coordinates of the outermost interval as 
        #keys and a list of all interval pairs as values
        nestedInts = {}
        
        #...cluster nested intervals together
        for i in sorted(intCoords[r], key=lambda i: i[1] - i[0] + 1, reverse=True):
            clusterFound = False
            
            for c in nestedInts:
                #Check if the current interval is covered by the outermost interval of this cluster already
                if i[0] >= c[0] and i[1] <= c[1]:
                    clusterFound = True
                    break
                
            #Check if we could not find a matching cluster for this interval
            if not clusterFound:
                #Create a new cluster
                nestedInts[i] = []
                
        #Testing
#         print("nestedInts:", nestedInts)
            
        #Assign each interval to every cluster whose outermost interval contains it
        for i in range(len(intCoords[r])):
            for c in nestedInts:
                if intCoords[r][i][0] >= c[0] and intCoords[r][i][1] <= c[1]:
                    nestedInts[c].append((i, intCoords[r][i]))
            
        #Testing
#         print("nestedInts:", nestedInts)
        
        #Select the best fitting interval of each cluster
        readLength = len(rdSeqs[f"s_{r}"].seq)
        bestFittingIndexes = []

        for c in nestedInts:
            minDiff = maxsize
            bestIndex = -1
                
            for i in nestedInts[c]:
                diff = abs((i[1][0] - i[1][1]) - readLength)
                    
                if diff < minDiff:
                    minDiff = diff
                    bestIndex = i[0]
                    
            bestFittingIndexes.append(bestIndex)
            
        #Testing
#         print("bestFittingIndexes:", bestFittingIndexes)
            
        for i in unique(bestFittingIndexes):
            bestFitting[r].append(intCoords[r][i])
    
    return bestFitting

In [9]:
#This function returns a dictionary with integer-based read ids as keys and a list of start stop tupels of homology 
#intervals reported by FindThoms as values. Stop coordinates may be corrected in the way that the k-1 overlap of the
#last k-mer part of the alpha-homology is not included anymore.
#Parameter: 
#res: Result of FindThoms as returned from function loadFindThomsRes
#corrEnd: Flag indicating if the end coordinate shall be adaptated
def parseCoordsFromFindThomsRes(res, corrEnd):
    ints = {}
        
    for r in res:
        if corrEnd:
            #Start position was corrected inside loadFindThomsRes already
            ints[r] = [(h["start"], h["end"] + 1 - 15) for h in res[r]]
        else:
            ints[r] = [(h["start"], h["end"]) for h in res[r]]
        
    return ints

In [11]:
from Bio import SeqIO

def readFasta(path):
    return [r for r in SeqIO.parse(open(path, 'r'), "fasta")]

#This function calculates the hash of a bitwise k-mer representation. The function is influenced by the code of "The
#minimizer Jaccard estimator is biased and inconsistent." from Belbasi et al. (function "minimap2_hash(seed,v,mask)"
#in file "minimap2_hash_uncompiled.py").
def getHash(kmer, mask):
    u = kmer & mask
    u = ((~u) + (u << 21)) & mask # u = (u<<21)-(u+1) = 77594587*u-1
    u = u ^ (u >> 24)
    u = ((u + (u << 3)) + (u << 8)) & mask # u *= 265
    u = u ^ (u >> 14)
    u = ((u + (u << 2)) + (u << 4)) & mask # u *= 21
    u = u ^ (u >> 28)
    u = (u + (u << 31)) & mask # u *= 2147483649

    return u

In [13]:
#This function returns a dictionary of k-mers from the given reference sketch that are covered by the given results 
#of some tool per each read from the given set of read ids.
#refSk: A list of start positions of k-mers in a sequence that are chosen as being part of the sketch
#res: A dictionary with read ids as keys and lists of start and stop coordinate tuples as values
def getCovKmers(refSk, res):
    covKmers = {}
    #Sort start positions in sketch
    sortedRefSk = sorted(refSk)
    
    #Iterate over reads:
    for r in res:
        #Initialize dictionary for this read
        covKmers[r] = {}
        
        #If there are no results we are done for this read
        if len(res[r]) == 0:
            continue
            
        #Initialize a flag marking that we have reached the end of our result list
        endReached = False
        #Sort result coordinates for this read increasingly by start position
        coords = sorted(res[r], key=lambda e: e[0])
        #Initialize an index to know which result positions we consider currently (we start with the one that has 
        #the smallest start coordinate)
        i = 0
        
        #Iterate over sorted list of start positions
        for p in sortedRefSk:
            #We need to switch to a result which has a larger start position as long as the position of the refer-
            #ence sketch k-mer is larger than the end position of our current result
            while p > coords[i][1]:
                #Move to the next result
                i += 1
                
                #Check if there is another result at all
                if i >= len(coords):
                    endReached = True
                    break
                    
            #Check if we set the flag
            if endReached:
                break
                
            #Check if the k-mer starts before the current result
            if p < coords[i][0]:
                continue
                
            #If we have reached this point the k-mer must fall inside the result
            covKmers[r][p] = None
        
    return covKmers

In [14]:
K = 15

In [12]:
refSeq = readFasta("../simulations/genomes/t2thumanChrY.fasta")[0].seq
blKmers = {int(l): None for l in open("highAbundKmersMiniK15w10Lrgr100BtStrnds.txt", 'r')}
fltRfSkMiK15W10 = [k for k in calcMiniSketch(str(refSeq), K, 10) if k[2] not in blKmers]

In [16]:
#Load read sequences
p = "../simulations/reads/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.0009090909090909091_" + \
"sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10.fasta"
rdSeqs = {r.id: r for r in readFasta(p)}
fltRfSkMiKmrStPosK15W10 = [k[0] + 1 - K for k in fltRfSkMiK15W10 if k[2] not in blKmers]
#Load homology intervals
FindThomsNestResApprxMppng = {}
p = "../simulations/homologies/homologies_t2thumanChrY_sr0.000_dr0.001_i" + \
"0.001_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_rm20_k15_w10_c1_u1_de" + \
"0.088_in-231.159_blhighAbundKmersMiniK15w10Lrgr100BtStrnds_nested.txt"
FindThomsNestResApprxMppng[20] = loadFindThomsRes(p, fltRfSkMiK15W10)

KeyboardInterrupt: 

In [18]:
bestFitHomInts = {}
covKmersBestFitInts = {}
bestFitHomInts[20] = filterBestFittingInts(parseCoordsFromFindThomsRes(FindThomsNestResApprxMppng[20], True))
covKmersBestFitInts[20] = getCovKmers(fltRfSkMiKmrStPosK15W10, bestFitHomInts[20])

In [48]:
cnt = 0
unfltHoms = parseCoordsFromFindThomsRes(FindThomsNestResApprxMppng[20], True)

In [55]:
# testInts = [i for i in unfltHoms[57465] + unfltHoms[57701] + unfltHoms[44033]]
# print(sorted(testInts))

def filterBestFittingIntsImproved(ints):
    bestFittingIntervals = {}
    
    for r in ints:
        readLength = len(rdSeqs[f"s_{r}"].seq) + 1 - K
        bestFittingIntervals[r] = []
        
        for i in sorted(ints[r], key=lambda c: abs((c[1] - c[0] + 1) - readLength)):
            nested = False
            
            for b in bestFittingIntervals[r]:
                #Check if intervals are nested
                if (i[0] >= b[0] and i[1] <= b[1]) or (i[0] <= b[0] and i[1] >= b[1]):
                    nested = True
                    break
                    
            if not nested:
                bestFittingIntervals[r].append(i)
                
    return bestFittingIntervals
        
fltHoms = filterBestFittingIntsImproved(unfltHoms)

In [78]:
betterfltHoms = filterBestFittingIntsBetter(unfltHoms)

In [79]:
nbhoms = 0

for r in betterfltHoms:
    nbhoms += len(betterfltHoms[r])
    
print("Number of best fitting homologies:", nbhoms)
print(len(betterfltHoms))

Number of best fitting homologies: 1300059
32295


In [80]:
cnt = 0

for r in bestFitHomInts[20]:
    if len(bestFitHomInts[20][r]) != len(betterfltHoms[r]):
        print("Read", r)
        print("Old:", bestFitHomInts[20][r])
        print("New:", betterfltHoms[r])
        cnt += 1
        
        if cnt == 3:
            break

Read 6198
Old: [(20991557, 21001331), (20991484, 21001381), (20991479, 21001245), (20991176, 21001197), (20991170, 21001192), (20991165, 21001165), (20991104, 21001123), (20991096, 21001118), (20991091, 21001116), (20991089, 21001094), (20991067, 21001084), (20991057, 21001070), (20990661, 21000689), (20990660, 21000655), (20989329, 21000606), (20989325, 21000605), (20989324, 21000599), (20989318, 21000590), (20989309, 21000589), (20989308, 21000579), (20989234, 21000505), (20989003, 21000532), (20989001, 21000492)]
New: [(20991557, 21001245), (20991176, 21001197), (20991170, 21001192), (20991165, 21001165), (20991104, 21001123), (20991096, 21001118), (20991091, 21001116), (20991089, 21001094), (20991067, 21001084), (20991057, 21001070), (20990661, 21000689), (20990660, 21000655), (20989329, 21000606), (20989325, 21000605), (20989324, 21000599), (20989318, 21000590), (20989309, 21000589), (20989308, 21000579), (20989234, 21000505)]
Read 12767
Old: [(12587315, 12604914), (12586432, 1260

In [84]:
unfltHoms[65681]

[(11213505, 11234615),
 (11213505, 11234571),
 (11213505, 11234527),
 (11213505, 11234517),
 (11213505, 11234395),
 (11213505, 11234385),
 (11213505, 11234368),
 (11213505, 11234348),
 (11213505, 11234346),
 (11213505, 11234343),
 (11213505, 11234338),
 (11213505, 11234178),
 (11213505, 11234149),
 (11213505, 11234137),
 (11213505, 11234129),
 (11213505, 11234122),
 (11213505, 11234117),
 (11213505, 11234116),
 (11213505, 11234082),
 (11213505, 11234027),
 (11213505, 11233989),
 (11213505, 11233988),
 (11213505, 11233982),
 (11213505, 11233973),
 (11213505, 11233948),
 (11213505, 11233945),
 (11213505, 11233944),
 (11213505, 11233930),
 (11213505, 11233928),
 (11213505, 11233837),
 (11213505, 11233756),
 (11213505, 11233746),
 (11213505, 11233645),
 (11213505, 11233638),
 (11213505, 11233633),
 (11213505, 11233575),
 (11213505, 11233564),
 (11213505, 11233495),
 (11213505, 11233491),
 (11213505, 11233471),
 (11213505, 11233466),
 (11213505, 11233397),
 (11213505, 11233359),
 (11213505,

In [83]:
filterBestFittingIntsBetter({65681: unfltHoms[65681]})

nestedInts: {(11213505, 11234615): [], (11211295, 11232357): [], (11212287, 11233284): [], (11209170, 11229888): []}
nestedInts: {(11213505, 11234615): [(0, (11213505, 11234615)), (1, (11213505, 11234571)), (2, (11213505, 11234527)), (3, (11213505, 11234517)), (4, (11213505, 11234395)), (5, (11213505, 11234385)), (6, (11213505, 11234368)), (7, (11213505, 11234348)), (8, (11213505, 11234346)), (9, (11213505, 11234343)), (10, (11213505, 11234338)), (11, (11213505, 11234178)), (12, (11213505, 11234149)), (13, (11213505, 11234137)), (14, (11213505, 11234129)), (15, (11213505, 11234122)), (16, (11213505, 11234117)), (17, (11213505, 11234116)), (18, (11213505, 11234082)), (19, (11213505, 11234027)), (20, (11213505, 11233989)), (21, (11213505, 11233988)), (22, (11213505, 11233982)), (23, (11213505, 11233973)), (24, (11213505, 11233948)), (25, (11213505, 11233945)), (26, (11213505, 11233944)), (27, (11213505, 11233930)), (28, (11213505, 11233928)), (29, (11213505, 11233837)), (30, (11213505, 1

{65681: [(11213505, 11229931), (11213015, 11229770)]}

In [20]:
ofile = open("covKmersBestFitIntsRr20.txt", 'w')

for r in covKmersBestFitInts[20]:
    ofile.write(f"s_{r}\n")
    
    for k in covKmersBestFitInts[20][r]:
        ofile.write(str(k) + '\n')
        
ofile.close()

# Covered *k*-mer Calculations for Edlib Results

[Back to Contents](#Contents)

Here, we generate sets of covered *k*-mers per read for edlib results using different distance thresholds.

In [27]:
from glob import glob

#Load edlib results
edlibRes = {10: {0.02: {}, 0.01: {}}, 20: {0.02: {}, 0.01: {}}}

for f in glob("../simulations/mappingDists/*edst"):
    rdid = int(f.split("_s_")[1].split(".edst")[0])
    p = "../simulations/edlibMappings/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.0009090" + \
    f"909090909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_ri{rdid}.wotD"
    alignments = []
    distances = []
    edlibRes[20][0.02][rdid] = []
    edlibRes[20][0.01][rdid] = []
    
    for l in open(p, 'r'):
        alignments.append(l.split(' '))
        
    for l in open(f"../simulations/mappingDists/sub_s_{rdid}.edst", 'r'):
        distances.append(int(l))
        
    #Sanity check
    if len(alignments) != len(distances):
        print("Number of loaded distances and number of loaded alignments is not the same for read", rdid)
        break
        
    for i in range(len(alignments)):
        if distances[i] <= 0.02 * len(rdSeqs[f"s_{rdid}"].seq):
            edlibRes[20][0.02][rdid].append(list(alignments[i]))
            edlibRes[20][0.02][rdid][-1].append(distances[i])
            
            if distances[i] <= 0.01 * len(rdSeqs[f"s_{rdid}"].seq):
                edlibRes[20][0.01][rdid].append(list(alignments[i]))
                edlibRes[20][0.01][rdid][-1].append(distances[i])
            
    if len(alignments) <= 10:
        edlibRes[10][0.02][rdid] = list(edlibRes[20][0.02][rdid])
        edlibRes[10][0.01][rdid] = list(edlibRes[20][0.01][rdid])

KeyError: 0.02

In [28]:
#Sanity check
print("len(edlibRes[20][0.02]):", len(edlibRes[20][0.02]))
print("len(edlibRes[20][0.01]):", len(edlibRes[20][0.01]))
print("len(edlibRes[10][0.02]):", len(edlibRes[10][0.02]))
print("len(edlibRes[10][0.01]):", len(edlibRes[10][0.01]))

len(edlibRes[20][0.02]): 32295
len(edlibRes[20][0.01]): 32295
len(edlibRes[10][0.02]): 31372
len(edlibRes[10][0.01]): 31372


In [29]:
#This function creates a dictionary with integer-based read ids as keys and lists of alignment coordinates as values
#from a given dictionary of edlib alignment results
#Parameter: res: Dictionary of edlib alignment results with integer-based read ids as keys and lists of alignment
#                infos parsed from file
def getEdlibResAlnCrds(res):
    return {r: [(int(rs[0]), int(rs[1])) for rs in res[r]] for r in res}

In [32]:
#Generate covered k-mer sets
covKmersEdlib = {10: {0.02: {}, 0.01: {}}, 20: {0.02: {}, 0.01: {}}}

for t in covKmersEdlib[20]:
    covKmersEdlib[20][t] = getCovKmers(fltRfSkMiKmrStPosK15W10, getEdlibResAlnCrds(edlibRes[20][t]))
    
    for r in edlibRes[10][t]:
        covKmersEdlib[10][t][r] = covKmersEdlib[20][t][r]
    
    for s in covKmersEdlib:
        ofile = open(f"covKmersEdlibRr{s}_{t}.txt", 'w')
        
        for r in covKmersEdlib[s][t]:
            ofile.write(f"s_{r}\n")
            
            for k in covKmersEdlib[s][t][r]:
                ofile.write(str(k) + '\n')
                
        ofile.close()

# Adding Missing Edit Distances to Edlib Result Files

[Back to Contents](#Contents)

In [33]:
p = "../simulations/edlibMappings/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.000909090909" + \
"0909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_ri*.wotD"

for f in glob(p):
    r = int(f.split("_ri")[1].split(".wotD")[0])
    fastDistances = []
    slowDistances = []
    
    for l in open(f"../simulations/mappingDists/sub_s_{r}.edst", 'r'):
        fastDistances.append(int(l))
        
    for l in open(f.replace("wotD", "er"), 'r'):
        slowDistances.append(int(l.split(' ')[-1]))
        
    if len(fastDistances) != len(slowDistances):
        print("Slow and fast distances are not the same!")
        print("Trouble causing read file is", f)
        break
        
    equal = True
        
    for i in range(len(fastDistances)):
        if fastDistances[i] != slowDistances[i]:
            print("Slow and fast distances are not the same!")
            print("Trouble causing read file is", f)
            equal = False
            break
            
    if not equal:
        break

In [103]:
from os.path import exists

#Generate a list of edlib result file names in which edit distances are yet missing
p = "../simulations/edlibMappings/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.000909090909" + \
"0909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_ri*.er"
oFile = open("missingDistanceFiles.txt", 'w')

for f in glob(p):
    if not exists(f.replace(".er", ".wotD")):
        oFile.write(f + '\n')
        
oFile.close()

In [4]:
from glob import glob

#Check if there is still edlib results where edit distances are missing
p = "../simulations/edlibMappings/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.000909090909" + \
f"0909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_ri*.er"
distanceMissing = False

for f in glob(p):
    for l in open(f, 'r'):
        if len(l.split(' ')) < 4:
            print("A distance is missing")
            distanceMissing = True
            break
            
    if distanceMissing:
        break

In [3]:
#Append edit distances to files where it is yet missing
for f in glob("/prj/homsketch/simulations/mappingDists/sub_s_*.edst"):
    #Get read id
    readId = int(f.split("_s_")[1].split(".edst")[0])
    #Read distances
    distances = []
    
    for l in open(f, 'r'):
        distances.append(int(l))
    
    #Open edlib file
    p = "../simulations/edlibMappings/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.000909090909" + \
f"0909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_ri{readId}.er"
    augmentedLines = []
    
    for l in open(p):
        #Append a distance if it is not already present
        

NameError: name 'glob' is not defined

# Covered *k*-mer Calculations for All Results of Competitors

[Back to Contents](#Contents)

In [89]:
import gzip

def parseCompressedPAF(filepath):
    resPerRead = {}
    
    for l in gzip.open(filepath, 'rt'):
        elems = l.strip().split('\t')
        rdid = int(elems[0].split('_')[1])
        
        if rdid in resPerRead:
            resPerRead[rdid].append((int(elems[7]), int(elems[8]) - 1))
        else:
            resPerRead[rdid] = [(int(elems[7]), int(elems[8]) - 1)]
            
    return resPerRead

In [90]:
pafRes = {}
covKmersApprxMppng = {}

for t in ["minimap2", "Winnowmap2"]:
    p = f"../simulations/{t}Res/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i" + \
    "0.0009090909090909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_k15.paf.gz"
    pafRes[t] = parseCompressedPAF(p)
    #Only consider results of relevant reads to speed this up
    covKmersApprxMppng[t] = getCovKmers(fltRfSkMiKmrStPosK15W10, pafRes[t])

In [91]:
for t in ["minimap2", "Winnowmap2"]:
    ofile = open(f"allPafRes_{t}.txt", 'w')
    
    for r in covKmersApprxMppng[t]:
        ofile.write(f"s_{r}\n")
        
        for k in covKmersApprxMppng[t][r]:
            ofile.write(str(k) + '\n')
            
    ofile.close()

In [107]:
cnt = 0
dirStr = ""


for r in range(69335, 69401):
    limitReached = False
    
    for d in glob(f"/prj/homsketch/simulations/blastRes/s_{r}/*"):
        cnt += 1
        dirStr += d + ' '
    
        if cnt % 100 == 1 and not limitReached:
            firstReadId = f"s_{r}"
            
        if cnt % 100 == 0:
            lastReadId = f"s_{r}"
            limitReached = True
        
    if limitReached:
        #Testing
#         print(f"list_{firstReadId}-{lastReadId}.txt")

        ofile = open(f"list_{firstReadId}-{lastReadId}.txt", 'w')
        ofile.write(dirStr + '\n')
        ofile.close()
        dirStr = ""
        cnt = 0
        
ofile = open(f"list_{firstReadId}-{lastReadId}.txt", 'w')
ofile.write(dirStr + '\n')
ofile.close()